In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
        'axes.linewidth':2.5, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['cyan','magenta','grey','crimson'], 4)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [4]:
pre_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\01_D&P_Osc_v1hpc\units\redo_brain_regions\V1HPC_pre_psth.pkl")
post_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\01_D&P_Osc_v1hpc\units\redo_brain_regions\V1HPC_post_psth.pkl")
novel_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\01_D&P_Osc_v1hpc\units\redo_brain_regions\V1HPC_novel_psth.pkl")

In [12]:
# combine the pre/post df with the novel df
psth_df = pd.concat([pre_df,post_df,novel_df])

In [13]:
def label_region(row):
    if (row['depth'] <= 3100) & (row['depth'] >= 2000):
        return 'v1'
    elif (row['depth'] < 1800) & (row['depth'] >= 600):
        return 'hippo'
    elif (row['depth'] < 600):
        return 'thal'
    else:
        return 'none'

In [ ]:
psth_df['region'] = psth_df.apply(lambda row: label_region(row), axis=1)

psth_df.head()

In [15]:
# convert depth column to insertion (ie flip 0 value from end of probe to surface of brain)
psth_df['insert'] = abs(psth_df['depth']-3100)


In [ ]:
print(psth_df.et.unique())
print(psth_df.region.unique())
print(psth_df.stim.unique())

---

In [17]:
psth2_df = psth_df.drop(['path', 'cc'], axis=1)
psth2_df = psth2_df[(psth2_df['region']=='v1') | (psth2_df['region']=='hippo')]
key_pca = 'r_groups'
num_groups = 2

In [ ]:
psth2_df.region.unique()

In [ ]:
for d,dd in psth_df.groupby(['stim']):
    

In [ ]:
my_stim = 'post'

solo_stim = psth_df[psth_df['stim'] == my_stim]

only_v1 = solo_stim[solo_stim['region'] == 'v1']
only_hpc = solo_stim[solo_stim['region'] == 'hippo']

In [ ]:


only_v1_kmeans = mz_ena.unit_kmeans(only_v1, num_groups, key_pca, 50, 250) #PCA on 0.5 to 2.5s

### Determine the number of mice in each situation

In [ ]:
# stim_list = ['pre', 'post', 'novel']
# region_ls = ['v1', 'hippo']

# for stim in stim_list:
#     foo1 = psth_df[psth_df['stim'] == stim]
#     print(stim)
#     for reg in region_ls:
#         foo2 = foo1[foo1['region'] == reg]
#         print(reg)
#         print(foo2.et.unique())
#     print('~~~~~~~~~~')

# Kmeans clustering

In [ ]:
print(psth_df.stim.unique())
print(psth_df.region.unique())
psth_df.head(2)

In [12]:
my_stim = 'post'

solo_stim = psth_df[psth_df['stim'] == my_stim]

only_v1 = solo_stim[solo_stim['region'] == 'v1']
only_hpc = solo_stim[solo_stim['region'] == 'hippo']

In [ ]:
key_pca = 'r_groups'
num_groups = 10

only_v1_kmeans = mz_ena.unit_kmeans(only_v1, num_groups, key_pca, 50, 250) #PCA on 0.5 to 2.5s
# only_hpc_kmeans = mz_ena.unit_kmeans(only_hpc, num_groups, key_pca, 50, 250) #PCA on 0.5 to 2.5s


### Plotting some spike rasters

In [ ]:
spk_pre_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\Multi_brain_regions\redo_brain_regions\V1HPC_pre_spikes.pkl")
spk_post_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\Multi_brain_regions\redo_brain_regions\V1HPC_post_spikes.pkl")
spk_novel_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\Multi_brain_regions\redo_brain_regions\V1HPC_novel_spikes.pkl")

# combine the pre/post df with the novel df
spike_df = pd.concat([spk_pre_df,spk_post_df,spk_novel_df])
spike_df.head()

In [19]:
stim_spk = spike_df[spike_df['stim'] == my_stim]

spike_v1 = stim_spk[stim_spk['region'] == 'v1']
spike_hpc = stim_spk[stim_spk['region'] == 'hippo']

In [ ]:
v1_map = dict(only_v1_kmeans[['cuid', 'r_groups']].values)
spike_v1['r_groups'] = spike_v1.cuid.map(v1_map)

hpc_map = dict(only_hpc_kmeans[['cuid', 'r_groups']].values)
spike_hpc['r_groups'] = spike_hpc.cuid.map(hpc_map)


In [ ]:
# rg is the r_groups label -- 0 to the num_groups
v1_rg = 9

f, ax = plt.subplots(1,2, sharex=False, sharey=False, figsize = (15,2))

g1 = sns.heatmap(only_v1_kmeans[only_v1_kmeans.r_groups==v1_rg].pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                 cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False, vmax=5, center = 2, vmin = -1,
                 robust = True, cbar=False)
ax[0].set(xlabel='Time (s)', ylabel=len(only_v1_kmeans[only_v1_kmeans.r_groups==v1_rg]['cuid'].unique()))
ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
ax[0].set_title('group ' + str(v1_rg), loc = 'center')

g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = only_v1_kmeans[only_v1_kmeans.r_groups==v1_rg], 
                  color = 'grey', ax=ax[1], ci=None)
ax[1].set(xlabel='Time (s)', ylabel='')
ax[1].set_title('group ' + str(v1_rg), loc = 'center')
ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
ax[1].xaxis.set_major_locator(MaxNLocator(7))

plt.show()

In [ ]:
rg_spike_v1 = spike_v1[spike_v1['r_groups'] == v1_rg]
for i in rg_spike_v1.cuid.unique():
    f, ax1 = plt.subplots(1,1, sharex='col', figsize=(10, 3))
    plt.plot( rg_spike_v1[(rg_spike_v1.cuid == i)].trial_spikes,  
             rg_spike_v1[(rg_spike_v1.cuid == i)].trial, 
             '.', color='grey')
    plt.title('unit: {0} --- rg: {1}'.format(str(i),v1_rg))
    plt.axvspan(0.5,0.7, color='grey', alpha=0.5)
    plt.ylabel('Trial')
    plt.xlim([0,2.5])
    plt.show()

In [ ]:
# rg is the r_groups label -- 0 to the num_groups
hpc_rg = 9

f, ax = plt.subplots(1,2, sharex=False, sharey=False, figsize = (15,2))

g1 = sns.heatmap(only_hpc_kmeans[only_hpc_kmeans.r_groups==hpc_rg].pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                 cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False, vmax=5, center = 2, vmin = -1,
                 robust = True, cbar=False)
ax[0].set(xlabel='Time (s)', ylabel=len(only_hpc_kmeans[only_hpc_kmeans.r_groups==hpc_rg]['cuid'].unique()))
ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
ax[0].set_title('group ' + str(hpc_rg), loc = 'center')

g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = only_hpc_kmeans[only_hpc_kmeans.r_groups==hpc_rg], 
                  color = 'crimson', ax=ax[1], ci=None)
ax[1].set(xlabel='Time (s)', ylabel='')
ax[1].set_title('group ' + str(hpc_rg), loc = 'center')
ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
ax[1].xaxis.set_major_locator(MaxNLocator(7))

plt.show()

In [ ]:
rg_spike_hpc = spike_hpc[spike_hpc['r_groups'] == hpc_rg]
for i in rg_spike_hpc.cuid.unique():
    f, ax1 = plt.subplots(1,1, sharex='col', figsize=(10, 3))
    plt.plot( rg_spike_hpc[(rg_spike_hpc.cuid == i)].trial_spikes,  
             rg_spike_hpc[(rg_spike_hpc.cuid == i)].trial, 
             '.', color='crimson')
    plt.title('unit: {0} --- rg: {1}'.format(str(i),hpc_rg))
    plt.axvspan(0.5,0.7, color='grey', alpha=0.5)
    plt.ylabel('Trial')
    plt.xlim([0,3.0])
    plt.show()

# V1 units

In [ ]:
plt.rcdefaults()

In [117]:
# For r_groups time interval [50, 250] and 10 groups
# v1_unit_spikes = ['et3832_425', 'et2_339', 'et383_433', 'et323_350', 'et3112_428', 
#                   'et3832_461', 'et383_406', 'et3832_447', 'et3112_382', 'et311_365']
# hpc_unit_spikes = ['et311_153', 'et1_315', 'et383_221', 'et383_171', 'et3832_262', 
#                    'et383_155', 'et323_183', 'et3112_222', 'et200_219', 'et2_302']

# For redoing the region ranges, r_groups time interval [50, 250], and 10 groups
v1_unit_spikes = ['et3832_425', 'et10_362', 'et10_375', 'et20_384', 'et383_411', 
                  'et2_428', 'et383_401', 'et2_389', 'et383_470', 'et311_405']
hpc_unit_spikes = ['et200_305', 'et383_205', 'et3832_200', 'et383_213', 'et10_207', 
                   'et2_179', 'et2_195', 'et200_183', 'et2_219', 'et10_193']


pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\v2'
to_plot = 1

In [ ]:
for i in range(num_groups): #make new plot for each r_group from the clustering algorithm above
    f, ax = plt.subplots(1,4, sharex=False, sharey=False, figsize = (25,4))
    ax = ax.ravel()
    # cbar_ax = f.add_axes([.91, .55, .02, .30])
    sns.set_style("ticks")
    
    plt_data = only_v1_kmeans[only_v1_kmeans.r_groups==i]
    plt_color = 'cyan'
    
    g1 = sns.heatmap(plt_data.pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                     cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False,
                     vmax=5, center=2, vmin=-1, robust=True, cbar_kws=dict(ticks=[-1,0,1,2,3,4,5]))
    ax[0].set(xlabel='Time (s)', ylabel=len(plt_data['cuid'].unique()))
    ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
    ax[0].set_title('group ' + str(i), loc = 'center')
        
    g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = plt_data, 
                      color = plt_color, ax=ax[1], ci=None)
    ax[1].set(xlabel='Time (s)', ylabel='ztc')
    ax[1].set_title('group ' + str (i), loc = 'center')
    ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
    ax[1].xaxis.set_major_locator(MaxNLocator(7))
    ax[1].tick_params(axis='x', labelrotation=0)
    ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
    ax[1].yaxis.set_major_locator(MaxNLocator(3))
    
    rg_spike_v1 = spike_v1[spike_v1['r_groups'] == i]
    unit_df = rg_spike_v1[rg_spike_v1['cuid'] == v1_unit_spikes[i]]
    g3 = ax[2].plot(unit_df.trial_spikes,  unit_df.trial, '.', color=plt_color)
    ax[2].set_title('group ' + str (i), loc = 'center')
    ax[2].set(xlabel='Time (s)', ylabel='Trial')
    ax[2].xaxis.set_major_locator(MaxNLocator(7))
    ax[2].set_yticks([0,10,20,30])
    ax[2].set_xlim([-0.1, 3.1])
    
    #include depth plot
    g4 = sns.histplot(data=plt_data, x='insert', stat='frequency', binwidth=80, 
                      kde=True, kde_kws={'bw_adjust': 2}, color=plt_color,
                      ax=ax[3])
    ax[3].set(xlabel='Depth (um)', ylabel='Frequency')
    ax[3].set_ylim([0, 55])
    ax[3].set_yticks([0,25,50])
    
    # Change the end of this next line with a new file name!!
    if to_plot == 1:
        fig_title = str('v1_rg'+str(i)+plt_color+'.pdf')
        out_title = pathway+'\\'+fig_title
        print(out_title)
        plt.savefig(out_title, transparent=True)
    
    plt.show()

# Hippocampus units

In [ ]:
for i in range(num_groups): #make new plot for each r_group from the clustering algorithm above
    f, ax = plt.subplots(1,4, sharex=False, sharey=False, figsize = (25,4))
    ax = ax.ravel()
#     cbar_ax = f.add_axes([.91, .55, .02, .30])
    sns.set_style("ticks")
    
    plt_data = only_hpc_kmeans[only_hpc_kmeans.r_groups==i]
    plt_color = 'crimson'
    
    g1 = sns.heatmap(plt_data.pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                     cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False,
                     vmax=5, center=2, vmin=-1, robust=True, cbar_kws=dict(ticks=[-1,0,1,2,3,4,5]))
    ax[0].set(xlabel='Time (s)', ylabel=len(plt_data['cuid'].unique()))
    ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
    ax[0].set_title('group ' + str(i), loc = 'center')
        
    g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = plt_data, 
                      color = plt_color, ax=ax[1], ci=None)
    ax[1].set(xlabel='Time (s)', ylabel='ztc')
    ax[1].set_title('group ' + str (i), loc = 'center')
    ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
    ax[1].xaxis.set_major_locator(MaxNLocator(7))
    ax[1].tick_params(axis='x', labelrotation=0)
    ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
    ax[1].yaxis.set_major_locator(MaxNLocator(3))
    
    rg_spike_hpc = spike_hpc[spike_hpc['r_groups'] == i]
    unit_df = rg_spike_hpc[rg_spike_hpc['cuid'] == hpc_unit_spikes[i]]
    g3 = ax[2].plot( unit_df.trial_spikes,  unit_df.trial, '.', color=plt_color)
    ax[2].set_title('group ' + str (i), loc = 'center')
    ax[2].set(xlabel='Time (s)', ylabel='Trial')
    ax[2].xaxis.set_major_locator(MaxNLocator(7))
    ax[2].set_yticks([0,10,20,30])
    ax[2].set_xlim([-0.1, 3.1])
    
    #include depth plot
    g4 = sns.histplot(data=plt_data, x='insert', stat='frequency', binwidth=80, 
                      kde=True, kde_kws={'bw_adjust': 2}, color=plt_color,
                      ax=ax[3])
    ax[3].set(xlabel='Depth (um)', ylabel='Frequency')
    ax[3].set_title('group ' + str(i), loc='center')
    ax[3].set_ylim([0, 85])
    ax[3].set_yticks([0,25,50,75])
    
    # Change the end of this next line with a new file name!!
    if to_plot == 1:
        fig_title = str('hpc_rg'+str(i)+plt_color+'.pdf')
        out_title = pathway+'\\'+fig_title
        print(out_title)
        plt.savefig(out_title, transparent=True)
    
    plt.show()


## Plot all r_groups together

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\v2'

f, ax = plt.subplots(2,1, sharex=True, sharey=False, figsize = (10,6))
palette1 = sns.color_palette("mako", num_groups)
palette2 = sns.color_palette("rocket", num_groups)

g1 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', hue='r_groups', palette=palette1,
                  data = only_v1_kmeans, ci=None, ax=ax[0], legend=True)
g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', hue='r_groups', palette=palette2,
                  data = only_hpc_kmeans, ci=None, ax=ax[1], legend=True)

ax[0].legend(ncol=2)
ax[1].legend(ncol=2)
ax[0].set_ylabel('V1')
ax[1].set_ylabel('HPC')
ax[0].axvspan(0.5, 0.7, facecolor='grey', alpha=0.25)
ax[1].axvspan(0.5, 0.7, facecolor='grey', alpha=0.25)
ax[0].set_yticks([0,10,20,30])
ax[1].set_yticks([-4,0,4,8,12])
ax[0].set_ylim([-5,31])
ax[1].set_ylim([-5,14])
ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
plt.xlabel('Time (s)')

if to_plot == 1:
    fig_title = str('all_combo_lineplot.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

---

# Save df of all r_groups

In [ ]:
only_v1_kmeans.head(2)

In [ ]:
only_hpc_kmeans.head(2)

In [115]:
new_path = r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\dataFrames/"

v1_path = new_path + 'v1_allrg_v2.pkl'
hpc_path = new_path + 'hpc_allrg_v2.pkl'

only_v1_kmeans.to_pickle(v1_path)
only_hpc_kmeans.to_pickle(hpc_path)

---

# Second iteration of kmeans to bring out the oscillator units

In [ ]:
v1_osc_ls = [3,7]
hpc_osc_ls = [2]

In [ ]:
v1_oscil = only_v1_kmeans['r_groups'].isin(v1_osc_ls)
v1_oscil = only_v1_kmeans[v1_oscil]
hpc_oscil = only_hpc_kmeans['r_groups'].isin(hpc_osc_ls)
hpc_oscil = only_hpc_kmeans[hpc_oscil]

In [ ]:
key_pca2 = 'r2_groups'
num_groups2 = 4

v1_oscillators_kmeans = mz_ena.unit_kmeans(v1_oscil, num_groups2, key_pca2, 50, 250) #PCA on 0.5 to 2.5s
hpc_oscillators_kmeans = mz_ena.unit_kmeans(hpc_oscil, num_groups2, key_pca2, 50, 250) #PCA on 0.5 to 2.5s

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\kmeans2'

for i in range(num_groups2): #make new plot for each r_group from the clustering algorithm above
    f, ax = plt.subplots(1,2, sharex=False, sharey=False, figsize = (15,3))
    ax = ax.ravel()
    # cbar_ax = f.add_axes([.91, .55, .02, .30])
    sns.set_style("ticks")
    
    g1 = sns.heatmap(v1_oscillators_kmeans[v1_oscillators_kmeans.r2_groups==i].pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                     cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False,
                     vmax=5, center=2, vmin=-1, robust=True, cbar_kws=dict(ticks=[-1,0,1,2,3,4,5]))
    ax[0].set(xlabel='Time (s)', ylabel=len(v1_oscillators_kmeans[v1_oscillators_kmeans.r2_groups==i]['cuid'].unique()))
    ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
    ax[0].set_title('group ' + str(i), loc = 'center')
        
    g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = v1_oscillators_kmeans[v1_oscillators_kmeans.r2_groups==i], 
                      color = 'grey', ax=ax[1], ci=None)
    ax[1].set(xlabel='Time (s)', ylabel='ztc')
    ax[1].set_title('group ' + str (i), loc = 'center')
    ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
    ax[1].xaxis.set_major_locator(MaxNLocator(7))
    ax[1].tick_params(axis='x', labelrotation=0)
    ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
    ax[1].yaxis.set_major_locator(MaxNLocator(3))
    
    if to_plot == 1:
        fig_title = str('v1_rg'+str(i)+'.pdf')
        out_title = pathway+'\\'+fig_title
        plt.savefig(out_title, transparent=True)
    
    plt.show()

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\kmeans2'

for i in range(num_groups2): #make new plot for each r_group from the clustering algorithm above
    f, ax = plt.subplots(1,2, sharex=False, sharey=False, figsize = (15,3))
    ax = ax.ravel()
    # cbar_ax = f.add_axes([.91, .55, .02, .30])
    sns.set_style("ticks")
    
    g1 = sns.heatmap(hpc_oscillators_kmeans[hpc_oscillators_kmeans.r2_groups==i].pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                     cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False,
                     vmax=5, center=2, vmin=-1, robust=True, cbar_kws=dict(ticks=[-1,0,1,2,3,4,5]))
    ax[0].set(xlabel='Time (s)', ylabel=len(hpc_oscillators_kmeans[hpc_oscillators_kmeans.r2_groups==i]['cuid'].unique()))
    ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
    ax[0].set_title('group ' + str(i), loc = 'center')
        
    g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = hpc_oscillators_kmeans[hpc_oscillators_kmeans.r2_groups==i], 
                      color = 'crimson', ax=ax[1], ci=None)
    ax[1].set(xlabel='Time (s)', ylabel='ztc')
    ax[1].set_title('group ' + str (i), loc = 'center')
    ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
    ax[1].xaxis.set_major_locator(MaxNLocator(7))
    ax[1].tick_params(axis='x', labelrotation=0)
    ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
    ax[1].yaxis.set_major_locator(MaxNLocator(3))
    
    if to_plot == 1:
        fig_title = str('hpc_rg'+str(i)+'.pdf')
        out_title = pathway+'\\'+fig_title
        plt.savefig(out_title, transparent=True)
    
    plt.show()

# Save the new r_groups dataFrames

In [ ]:
v1_oscillators_kmeans.head()

In [ ]:
hpc_oscillators_kmeans.head()

In [ ]:
new_path = r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\V1Hippo_spikes/"

v1_path = new_path + 'v1_rg_df.pkl'
hpc_path = new_path + 'hpc_rg_df.pkl'

v1_oscillators_kmeans.to_pickle(v1_path)
hpc_oscillators_kmeans.to_pickle(hpc_path)

# Choose which r_group to plot

In [ ]:
v1_rgroup = [0,1,2,3]
hpc_rgroup = [0]

# change the dataFrame depending on r_groups or r2_groups (only_v1_kmeans,only_hpc_kmeans or v1_oscillators_kmeans,hpc_oscillators_kmeans)
V1_rg = v1_oscillators_kmeans['r2_groups'].isin(v1_rgroup)
V1_rg = v1_oscillators_kmeans[V1_rg]
HPC_rg = hpc_oscillators_kmeans['r2_groups'].isin(hpc_rgroup)
HPC_rg = hpc_oscillators_kmeans[HPC_rg]
kmeans_all = pd.concat([V1_rg,HPC_rg])
kmeans_all = kmeans_all.sort_values(by=['r_groups'])

In [ ]:
is_sorted = 'yes'

v1_km = V1_rg.pivot('cuid', 'times', 'zscore')
hip_km = HPC_rg.pivot('cuid', 'times', 'zscore')

if is_sorted == 'yes':
    v1_km_np = v1_km.values[ np.argsort(np.mean(v1_km.values[:,50:80], axis = 1) )] #if I want sorted activity
    hip_km_np = hip_km.values[ np.argsort(np.mean(hip_km.values[:,50:80], axis = 1) )] #if I want sorted activity
elif is_sorted == 'no':
    v1_km_np = v1_km.values
    hip_km_np = hip_km.values

In [ ]:
sp_dims = (15, 4) #(width, height)
v_max = 5
v_min = -1
xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
y_min = -2
y_max = 6

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs'

f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=(15,4))
f.tight_layout()
plt.suptitle('kmeans: '+stim_cond)
plt.minorticks_on()

# Heatmap for A
sns.heatmap(v1_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel(str(len(V1_rg.cuid.unique())))
# ax1.set_xticklabels(xvalues, rotation=0)
ax1.set_xlim(0.4*100,1.6*100)
ax1.set_title('V1')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(hip_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel(str(len(HPC_rg.cuid.unique())))
# ax2.set_xticklabels(xvalues, rotation=0)
ax2.set_xlim(0.4*100,1.6*100)
ax2.set_title('HPC')
cbar = ax2.collections[0].colorbar
# cbar.ax.tick_params(labelsize=25)
cbar.set_label('Z-score')

# Change the end of this next line with a new file name!!
if to_plot == 1:
    fig_title = str('V1HPC_oscillators_heat.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)
    
plt.show()

In [ ]:
sp_dims = (10, 4) #(width, height)
f, ax = plt.subplots(1,1, sharex='col', figsize=sp_dims)

#Combo lineplot
sns.lineplot(x = 'times', y = 'ztc', 
             estimator = 'mean', 
             data = kmeans_all, 
             hue = 'region',
             hue_order = ['v1','hippo'],
             legend=False,
             ci=None) #ci is the error

plt.legend(['V1','HPC'], bbox_to_anchor=(1.1, 1.05))

plt.axvspan(0.5, 0.7, color='lightgrey', alpha = 0.5)
plt.axhline(y=0, xmin=0, xmax=3, c='black', linewidth=1, zorder=0)

plt.xlim(0.4,1.6)
plt.xticks([0.4,0.6,0.8,1,1.2,1.4,1.6])
# plt.yticks([-4,0,4,8,12,16,20])
# plt.ylim([-4,22])
plt.minorticks_on()
plt.xlabel('Time (s)')
plt.ylabel('zscore')
plt.title('kmeans: '+stim_cond)

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs'
if to_plot == 1:
    fig_title = str('V1HPC_oscillators_line.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)
    
plt.show()

---

---

---